PART 1 : Q2_a

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=cc33242fcd5e5f13ac894a8dbdaca51df1bc50280cd77d887c66ce49bde11fd5
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
import pandas as pd
from pyspark import SparkConf, SparkContext

In [4]:
# set up the Spark context
conf = SparkConf().setAppName("ChannelPromotions").setMaster("local")
sc = SparkContext.getOrCreate()


# read in the data from the input file
data = sc.textFile("data.txt")

# map the channel IDs to (ID, 1) pairs
channel_counts = data.flatMap(lambda line: line.split()[1:]) \
                     .map(lambda channel_id: (channel_id, 1))

# reduce the channel counts by summing the values for each key
channel_totals = channel_counts.reduceByKey(lambda a, b: a + b)

# find the top 5 channels with the most link exchanges
top_channels = channel_totals.takeOrdered(5, key=lambda x: -x[1])


# print the results
for channel, count in top_channels:
    print(f"Channel {channel} has been advertised {count} times.")

Channel 859 has been advertised 1933 times.
Channel 5306 has been advertised 1741 times.
Channel 2664 has been advertised 1528 times.
Channel 5716 has been advertised 1426 times.
Channel 6306 has been advertised 1394 times.


PAR1 : Q2_b

In [5]:
def contribution(channel_id):

  # specify the channel to find promotion count for
  target_channel = channel_id
  # filter the data to only include transactions for the target channel
  target_data = data.filter(lambda line: target_channel in line.split()[1:])

  # map the channel IDs to (ID, 1) pairs
  channel_counts = target_data.flatMap(lambda line: line.split()[1:]) \
                              .map(lambda channel_id: (channel_id, 1))

  # reduce the channel counts by summing the values for each key
  channel_totals = channel_counts.reduceByKey(lambda a, b: a + b)

  # find the total promotion count for the target channel
  promotion_count = channel_totals.filter(lambda x: x[0] == target_channel).map(lambda x: x[1]).collect()

  # print the result
  print(f"The exchange count for channel {target_channel} is {promotion_count[0]}.")


contribution('1748')
contribution('5633')
contribution('3469')

The exchange count for channel 1748 is 130.
The exchange count for channel 5633 is 30.
The exchange count for channel 3469 is 119.
